# Проверка гипотез

In [48]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

### Подгрузим данные:

In [49]:
df = pd.read_csv('songs_normalize.csv')
df.head()

,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


### При выполнении проекта у нас появились 3 гипотезы, которые мы проверим ниже:

## Гипотеза 1: популярность песен с минорным ладом в среднем больше, чем популярность песен с мажорным.

Нулевая гипотеза ($Н_О$): средняя популярность песен с минорным ладом меньше или равна средней популярности песен с мажорным ладом.
Альтернативная гипотеза ($Н_A$): средняя популярность песен с минорным ладом больше средней популярности песен с мажорным ладом.

$$H_O: \mu_{минор} <= \mu_{мажор}$$
$$H_A: \mu_{минор} > \mu_{мажор}$$

, где $\mu_{минор}$ - ожидаемая популярность песен с минорным ладом, $\mu_{мажор}$ - ожидаемая популярность песен с мажорным ладом. 

Разделите данные по популярности песен с минорным (mode=0) и мажорным (mode=1) ладом

In [50]:
X_minor = df_tracks[df_tracks['mode'] == 0]['popularity']
Y_major = df_tracks[df_tracks['mode'] == 1]['popularity']

In [51]:
X_minor.shape, Y_major.shape

((893,), (1107,))

Можем сделать вывод о достаточном количестве данных, чтобы говорить о применимости ЦПТ. Проверим гипотезу на уровне $5\%$ с помощью одностороннего $t$-теста (применяем его, так как не знаем настоящую теоритическую дисперсию):

In [52]:
t_stat, p_value = stats.ttest_ind(X_minor, Y_major, alternative='greater')
print("t-статистика:", t_stat)
print("p-значение:", p_value)

t-статистика: 0.9546913966209316
p-значение: 0.16992463522881657


Вывод на основе p-значения

In [53]:
if p_value < 0.05:
    print("Отвергаем нулевую гипотезу в пользу альтернативной гипотезы: популярность песен с минорным ладом больше.")
else:
    print("Нет оснований отвергать нулевую гипотезу.")

Нет оснований отвергать нулевую гипотезу.


#### Вывод: наша первоначальная гипотеза о том, что средняя популярность песен с минорным ладом больше средней популярности песен с мажорным ладом, не подтвержается, так как нет оснований отвергать то, что средняя популярность песен с минорным ладом меньше или равна средней популярности песен с мажорным ладом

## Гипотеза 2: между уровнем громкости и мерой, которая показывает, насколько песня подходит для танцев есть положительная зависимость.

Нулевая гипотеза ($Н_О$): коэффициент корреляции между уровнем громкости и танцевальностью равен нулю (нет корреляции). \
Альтернативная гипотеза ($Н_A$): коэффициент корреляции между уровнем громкости и танцевальностью больше нуля (положительная корреляция).

$$H_O: \rho = 0 $$
$$Н_A: \rho > 0 $$

, где $\rho$ - коэффициент корреляции между уровнем громкости и танцевальностью. 

Вычислим наблюдаемый коэффициент корреляции Пирсона:

In [55]:
obs_corr = np.corrcoef(df_tracks['loudness'], df_tracks['danceability'])[0, 1]
obs_corr

-0.033314857853921305

Теперь для проверки выполним перестановочный тест:

In [56]:
num_permutations = 10000
permuted_corrs = []

for _ in range(num_permutations):
    permuted_loudness = np.random.permutation(df_tracks['loudness'])
    permuted_corr = np.corrcoef(permuted_loudness, df_tracks['danceability'])[0, 1]
    permuted_corrs.append(permuted_corr)

Вычислим критическое значение p-значения для уровня значимости $5\%$ $\rho_{crit}$:



In [57]:
p_crit = np.percentile(permuted_corrs, q=5)
p_crit

-0.037589908615190064

Выведем получившиеся значения:

In [58]:
print("Наблюдаемый коэффициент корреляции Пирсона:", obs_corr)
print("p_crit:", p_crit)

Наблюдаемый коэффициент корреляции Пирсона: -0.033314857853921305
p_crit: -0.037589908615190064


In [59]:
if obs_corr < p_crit:
    print("Отвергаем нулевую гипотезу в пользу альтернативной гипотезы: есть положительная зависимость.")
else:
    print("Нет оснований отвергать нулевую гипотезу.")

Нет оснований отвергать нулевую гипотезу.


#### Вывод: наша первоначальная гипотеза о том, что между уровнем громкости и мерой, которая показывает, насколько песня подходит для танцев есть положительная зависимость, не подтверждается, так как нет оснований отвергать, что коэффициент корреляции между уровнем громкости и танцевальностью равен нулю (нет корреляции). 

## Гипотеза 3: между длительностью песни и популярностью есть отрицательная зависимость. 

Нулевая гипотеза ($Н_О$): коэффициент корреляции между длительностью песни и популярностью равен нулю (нет корреляции).\
Альтернативная гипотеза ($Н_A$): коэффициент корреляции между длительностью песни и популярностью меньше нуля (отрицательная корреляция).

$$H_O: \rho = 0 $$
$$H_A: \rho < 0$$ \
, где $\rho$ — коэффициент корреляции между длительностью песни и популярностью.

Вычислим наблюдаемый коэффициент корреляции Пирсона:

In [60]:
obs_corr2 = np.corrcoef(df_tracks['duration_ms'], df_tracks['popularity'])[0, 1]
obs_corr2

0.05061721936219039

Теперь для проверки выполним перестановочный тест:

In [61]:
num_per = 10000
per_corrs = []

for _ in range(num_per):
    permuted_duration_ms = np.random.permutation(df_tracks['duration_ms'])
    per_corr = np.corrcoef(permuted_duration_ms, df_tracks['popularity'])[0, 1]
    per_corrs.append(per_corr)

Вычислим критическое значение p-значения для уровня значимости $5\%$ $\rho_{crit}$:

In [62]:
p2_crit = np.percentile(per_corrs, q=5)
p2_crit

-0.03627805781911337

In [63]:
print("Наблюдаемый коэффициент корреляции Пирсона:", obs_corr2)
print("p_crit:", p2_crit)

Наблюдаемый коэффициент корреляции Пирсона: 0.05061721936219039
p_crit: -0.03627805781911337


In [64]:
if obs_corr2 < p2_crit:
    print("Отвергаем нулевую гипотезу в пользу альтернативной гипотезы: есть положительная зависимость.")
else:
    print("Нет оснований отвергать нулевую гипотезу.")

Нет оснований отвергать нулевую гипотезу.


#### Вывод: наша первоначальная гипотеза о том, что между длительностью песни и популярностью есть отрицательная зависимость, не подтверждается, так как нет оснований отвергать, что коэффициент корреляции между длительностью песни и популярностью равен нулю.